## Training
* Trains a neural network with the given hyperparameters

In [ ]:
function train_epoch!(m, ps, opt; split::String="training", rng=Random.GLOBAL_RNG)
    epoch = get_epoch(split)
    sampling_order = get_sampling_order(epoch, split, rng)
    batchloss(x, y, z, w) = model_loss(m, x, y, z, w)
    nbatches = Int(ceil(length(sampling_order) / G.batch_size))
    for iter = 1:nbatches
        batch, _ = get_batch(epoch, iter, G.batch_size, sampling_order, split == "training")
        Flux.train!(batchloss, ps, [batch], opt)
        device_free!(batch)
    end
end;

In [ ]:
# trains a model with the given hyperparams and returns its validation loss
function train_model(
    hyp::Hyperparams;
    max_checkpoints::Int = 1000,
    epochs_per_checkpoint::Int = 1,
    patience::Int = 0,
    verbose::String = "",
    init_model = nothing,
)
    global G = hyp
    opt = get_optimizer(G.optimizer, G.learning_rate, G.optimizer_weight_decay)
    rng = Random.Xoshiro(G.seed)
    Random.seed!(rand(rng, UInt64))
    if CUDA.functional()
        Random.seed!(CUDA.default_rng(), rand(rng, UInt64))
        Random.seed!(CUDA.CURAND.default_rng(), rand(rng, UInt64))
    end
    
    if isnothing(init_model)
        m = build_model(rng=rng) |> device
    else
        m = init_model |> device
    end
    best_model = m |> cpu
    ps = Flux.params(m)
    stopper = early_stopper(max_iters = max_checkpoints, patience = patience, min_rel_improvement = 1e-3)
    
    losses = []
    loss = Inf
    while (!stop!(stopper, loss))
        for i = 1:epochs_per_checkpoint
            train_epoch!(m, ps, opt; rng=rng)
        end
        loss = split_loss(m, "validation"; rng=rng)
        push!(losses, loss)
        if loss == minimum(losses)
            best_model = m |> cpu
        end
        if verbose == "info"
            @info "losses: $loss"
        elseif verbose == "debug"
            training_loss = uncalibrated_split_loss(m, "training")
            validation_loss = uncalibrated_split_loss(m, "validation")            
            @info "losses: $training_loss $validation_loss $loss"
        end
    end
    global G = nothing
    epochs = stopper.iters - stopper.iters_without_improvement
    best_model, epochs, minimum(losses)
end;